In [13]:
import pandas as pd
orders=pd.read_csv("/content/orders.csv")
orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [14]:
import pandas as pd
users=pd.read_json("/content/users.json")
users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [15]:
import sqlite3
import pandas as pd

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Read the SQL script from the file
with open('/content/restaurants.sql', 'r') as f:
    sql_script = f.read()

# Execute the SQL script to create tables and insert data
conn.executescript(sql_script)

# Now, read the data from the 'restaurants' table
restaurants = pd.read_sql('SELECT * FROM restaurants', conn)

conn.close()
print(restaurants.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [16]:
df = orders.merge(users, on="user_id", how="left") \
.merge(restaurants, on="restaurant_id", how="left")
print(df.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [17]:
len(orders)

10000

In [18]:
df.shape

(10000, 12)

In [19]:
print(df.head())
print(df.info())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [20]:
print("Total Orders:",
      df['order_id'].nunique())

Total Orders: 10000


In [21]:
df['order_date'] = pd.to_datetime(df['order_date'])
orders_time = df.groupby(df['order_date'].dt.date)['order_id'].count()


/tmp/ipython-input-1595709784.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [22]:
print(df.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')


In [23]:
top_restaurants = df['restaurant_name_y'].value_counts().head(5)
print(top_restaurants)



restaurant_name_y
Restaurant_383    36
Restaurant_287    35
Restaurant_231    32
Restaurant_92     32
Restaurant_475    30
Name: count, dtype: int64


In [24]:
top_restaurants = df['name'].value_counts().head(5)
print(top_restaurants)

name
User_2973    13
User_1515    12
User_874     11
User_496     11
User_1337    11
Name: count, dtype: int64


In [25]:
df.rename(columns={'name': 'restaurant_name'}, inplace=True)

top_restaurants = df['restaurant_name'].value_counts().head(5)
print(top_restaurants)

restaurant_name
User_2973    13
User_1515    12
User_874     11
User_496     11
User_1337    11
Name: count, dtype: int64


In [26]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [27]:
print(df['user_id'].value_counts().head(5))


user_id
2973    13
1515    12
874     11
496     11
1337    11
Name: count, dtype: int64


In [28]:
print(df['total_amount'].describe())

count    10000.000000
mean       801.162412
std        405.458753
min        100.200000
25%        446.310000
50%        806.295000
75%       1149.227500
max       1499.830000
Name: total_amount, dtype: float64


In [29]:
print(round((df[df['membership'].str.lower()=='gold']['user_id'].nunique()/df['user_id'].nunique())*100, 2))

49.6


In [30]:
print(df.groupby('restaurant_name')['order_id'].count().sort_values(ascending=False).head(1))

restaurant_name
User_2973    13
Name: order_id, dtype: int64


In [31]:
print(df.groupby('restaurant_name_x')['order_id'].count().idxmax())

Spice Foods Non-Veg


In [32]:
gold_orders = df[df['membership']=='Gold']['order_id'].count()
print(gold_orders)

4987


In [33]:
round(df[df['city']=='Hyderabad']['total_amount'].sum())

1889367

In [34]:
df['user_id'].nunique()

2883

In [35]:
round(df[df['membership']=='Gold']['total_amount'].mean(), 2)

np.float64(797.15)

In [36]:
(df['rating']>= 4.5).sum()

np.int64(3374)

In [37]:
g = df[df['membership']=='Gold']
g[g['city']==g.groupby('city')['total_amount'].sum().idxmax()]['order_id'].count()

np.int64(1337)